<a href="https://colab.research.google.com/github/FajarKKP/ML-DL_related_project/blob/main/SIBI_ImageDataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import sklearn
import numpy as np

from sklearn.metrics import confusion_matrix, classification_report 
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.metrics import FalseNegatives, FalsePositives, TrueNegatives, TruePositives, Precision, Recall, Accuracy, AUC

In [ ]:
#Import data from Gdrive
data_dir = '/content/drive/MyDrive/Dataset_pakai'

# Initialize batch size and image size
batch_size = 32
image_size = (224,224)
cl_names= ["A","B","C","D","E","F","G","H","I","K",
           "L","M","N","O","P","Q","R","S","T","U",
           "V","W","X","Y",]

In [ ]:
#For Augmentation
train_datagen = ImageDataGenerator(
                    rotation_range=10,
                    horizontal_flip=True)
 


In [ ]:

train_generator = train_datagen.flow_from_directory(
        '/content/drive/MyDrive/testing_folder/train', 
        target_size=image_size, 
        batch_size=batch_size,
        shuffle = True,
        class_mode='sparse',
        classes=cl_names)
 
val_generator = test_datagen.flow_from_directory(
        '/content/drive/MyDrive/testing_folder/val', 
        target_size=image_size, 
        batch_size=batch_size,
        # shuffle = True,
        class_mode='sparse',
        classes=cl_names)

test_generator = ImageDataGenerator.flow_from_directory(
        '/content/drive/MyDrive/testing_folder/test', 
        target_size=image_size, 
        batch_size=batch_size,
        class_mode='sparse',
        classes=cl_names)

In [ ]:
# Import the EfficientNet model
eff_model1 = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, input_shape=(224,224,3))
print(type(eff_model1))
eff_model1.summary()

In [ ]:
# Freeze the EfficientNet, then set the dropout rate we will use for our model
eff_model1.trainable = False
dropout_rate = 0.1 #@param {type:"slider", min:0, max:1, step:0.1}

In [ ]:
# Create a new model, combining the EfficientNet and new output layer with 24 output class

model = models.Sequential()
model.add(eff_model1)
model.add(layers.GlobalMaxPooling2D(name="gap"))
# model.add(layers.Flatten(name="flatten"))
if dropout_rate > 0:
    model.add(layers.Dropout(dropout_rate, name="dropout_out"))

model.add(layers.Dense(24, activation="softmax", name="fc_out"))

model.summary()

In [ ]:
# Train model
epochs = 50

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs={}):
    if(logs.get('accuracy')>0.98):
      print("\n Good enough, its above 98%")
      self.model.stop_training=True
callbacks = myCallback()

# early_stopping = keras.callbacks.EarlyStopping(
#     monitor="accuracy",
#     patience=5,
  
# )


model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="sparse_categorical_crossentropy", #"sparse_categorical_crossentropy" if label is in integer
    metrics=['accuracy'],
)
final_model = model.fit(train_generator, epochs=epochs, callbacks=[callbacks], validation_data=val_generator)

In [ ]:
def plot_hist(final_model):
    plt.plot(final_model.history["accuracy"])
    plt.plot(final_model.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


plot_hist(final_model)

In [ ]:
model.save('/content/drive/MyDrive/Hasil_model_EfficientNet/final_model_generator.h5')
model.save('/content/drive/MyDrive/Hasil_model_EfficientNet/final_model_generator.hdf5')